In [5]:
import numpy as np 
import pandas as pd
import random

# For reproducibility
random.seed(42)
np.random.seed(42)

num_hours = 1008  # 42 days * 24 hours
patient_df = pd.DataFrame({'Absolute Hour': np.arange(1, num_hours + 1)})

# Derive time components
patient_df['Week'] = ((patient_df['Absolute Hour'] - 1) // 168) + 1
patient_df['Hour (Day)'] = ((patient_df['Absolute Hour'] - 1) % 24) + 1
patient_df['Hour (Week)'] = ((patient_df['Absolute Hour'] - 1) % 168) + 1

# --- Define parameters for the Floating pod's sinusoidal variation ---
# We want f(hour) to be around 80 at its peak and 49 at its trough.
# We use the model: 
#   f(hour) = 55 + 15.5 * ( sin(2*pi/24*(Hour - phase)) + offset_shift )
#
# Solve for offset_shift using the trough condition:
#   55 + 15.5*( -1 + offset_shift ) = 49  ⟹  offset_shift ≈ 0.6129
offset_shift = 0.6129
amplitude = 15.5

# Choose phase so that the peak occurs at the desired hour.
# For example, to have a peak at Hour ≈ 16:
#   2*pi/24*(16 - phase) = pi/2  ⟹  phase = 10
phase = 10

# Compute the time-dependent mean for the Floating pod without saving it to the DataFrame
floating_mean = 55 + amplitude * (
    np.sin(2 * np.pi / 24 * (patient_df['Hour (Day)'] - phase)) + offset_shift
)

# Generate the actual Floating count around this mean, with a standard deviation of 3
patient_df['Floating'] = np.clip(
    np.random.normal(loc=floating_mean, scale=3).round().astype(int),
    0, None
)

# Generate counts for the other areas as before
patient_df['Purple'] = np.clip(
    np.random.normal(loc=15, scale=3, size=len(patient_df)).round().astype(int),
    0, None
)
patient_df['EDOU'] = np.clip(
    np.random.normal(loc=5, scale=2, size=len(patient_df)).round().astype(int),
    0, None
)

# Optionally, save the DataFrame to a CSV file
patient_df.to_csv("patient_forecast.csv", index=False)
patient_df


,Absolute Hour,Week,Hour (Day),Hour (Week),Floating,Purple,EDOU
0,1,1,1,1,55,18,4
1,2,1,2,2,51,13,1
2,3,1,3,3,51,19,6
3,4,1,4,4,54,16,3
4,5,1,5,5,49,21,5
...,...,...,...,...,...,...,...
1003,1004,6,20,164,70,13,6
1004,1005,6,21,165,71,13,7
1005,1006,6,22,166,66,12,4
1006,1007,6,23,167,63,17,4
